In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [4]:
# optional to load different towns
#client.set_timeout(15)
client.load_world('Town04')

In [3]:
#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [4]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [5]:
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [6]:
# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [7]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

In [8]:
#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

#camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

In [12]:
import cv2
import os
import time  # Import the time module

# Create a folder to store the images if it doesn't exist
folder_name = "captured_images_verify"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

counter = 1  # Initialize counter for image naming

while True:
    # Assuming camera_data is being updated somewhere in the loop
    # Display with imshow
    cv2.imshow('All cameras', camera_data['image'])
    
    # Save the image
    image_name = os.path.join(folder_name, f"image_{counter}.jpg")
    cv2.imwrite(image_name, camera_data['image'])
    
    # Verify image received in terminal
    if os.path.exists(image_name):
        print(f"Image saved successfully: {image_name}")
    else:
        print("Error: Image not saved!")
    
    # Increment counter
    counter += 1
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()


Image saved successfully: captured_images_verify\image_1.jpg
Image saved successfully: captured_images_verify\image_2.jpg
Image saved successfully: captured_images_verify\image_3.jpg
Image saved successfully: captured_images_verify\image_4.jpg
Image saved successfully: captured_images_verify\image_5.jpg
Image saved successfully: captured_images_verify\image_6.jpg
Image saved successfully: captured_images_verify\image_7.jpg
Image saved successfully: captured_images_verify\image_8.jpg
Image saved successfully: captured_images_verify\image_9.jpg
Image saved successfully: captured_images_verify\image_10.jpg
Image saved successfully: captured_images_verify\image_11.jpg
Image saved successfully: captured_images_verify\image_12.jpg
Image saved successfully: captured_images_verify\image_13.jpg
Image saved successfully: captured_images_verify\image_14.jpg
Image saved successfully: captured_images_verify\image_15.jpg
Image saved successfully: captured_images_verify\image_16.jpg
Image saved succe

In [9]:
#grab a snapshot from the camera an show in a pop-up window
img = camera_data['image']
cv2.imshow('RGB Camera',img)
cv2.waitKey(0)

-1

In [10]:
# clean up after yourself

camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [7]:
len(spawn_points)

10422

In [8]:
start_point = spawn_points[0]
spectator = world.get_spectator()
spectator.set_transform(start_point)

In [12]:
spectator.set_transform(carla.Transform(carla.Location(x=-1085.286377, y=3112.225830, z=356.060608), carla.Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000)))

In [11]:
print(start_point)

Transform(Location(x=-1085.286377, y=3112.225830, z=356.060608), Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000))
